# Price Forecasting: Time Series Analysis and Model Training


### 1. Problem Statement
The energy industry is undergoing a transformative journey, marked by rapid modernization and technological advancements. Infrastructure upgrades, integration of intermittent renewable energy sources, and evolving consumer demands are reshaping the sector. However, this progress comes with its challenges. Supply, demand, and prices are increasingly volatile, rendering the future less predictable. Moreover, the industry's traditional business models are being fundamentally challenged. In this competitive and dynamic landscape, accurate decision-making is pivotal. The industry relies heavily on probabilistic forecasts to navigate this uncertain future, making innovative and precise forecasting methods essential that aids stakeholders in making strategic decisions amidst the shifting energy landscape. 

### 2. Data Ingestion

#### [Zenml-MLfow pipeline for Data Ingestion](../steps/price_forecasting/data_ingestion.py)

#### 2.1 Import Data and Required Packages
Importing Pandas, Numpy, Matplotlib,Scipy and Seaborn

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
import joblib
%matplotlib inline

#### 2.2 Import the CSV Data as Pandas DataFrame

In [ ]:
df_price = pd.read_csv('../dataset/Price forecasting/Price Forecasting data upto December 24.csv', sep=',')

### 3. Data Preprocessing

#### [Zenml-MLfow pipeline for Data Preprocessing](../steps/price_forecasting/data_preprocessing.py)

#### 3.1 Show top 5 records


In [ ]:
df_price.head()

#### 3.2 Shape of Dataset


In [ ]:
df_price.shape

#### 3.3 Dataset Information
<li>Delivery Day: Represents date</li>
<li>Hours: Represents Specific Hour of Day</li>
<li>Prices\n(Eur/MWh): Represents price of electricity in Euro per megawatt of electricity</li>

#### 3.4 Descriptive Statistics

In [ ]:
df_price.describe()

#### 3.5 Dealing with missing values


Checking NaN values

In [ ]:
df_price.isna().sum()

Dropping rows having values NaN

In [ ]:
df_price.dropna(how='all', inplace=True)
df_price.isna().sum()

#### 3.6 Dealing with Inconsistent Datas

<p>From descriptive statistics analysis, minimum value of price is found to be negative which concludes there are some negative values of price which makes no sense.</p>

<li>Checking For Negative or Zero Prices</li>

In [ ]:
df_price[df_price['Prices\n(EUR/MWh)']<=0]

<li>Replacing Negative or Zero Prices with NaN</li>

In [ ]:
mask = df_price['Prices\n(EUR/MWh)'] <= 0
df_price.loc[mask, 'Prices\n(EUR/MWh)'] = np.nan

<li>Using linear interpolation to fill NaN Values</li>

In [ ]:
df_price['Prices\n(EUR/MWh)'].interpolate(inplace=True)

#### 3.7 Removing Outliers

In [ ]:
zscore = scipy.stats.zscore(df_price['Prices\n(EUR/MWh)'])
df_price = df_price[abs(zscore)<5]

### 4. Data Visualization

#### 4.1 Datetime Parsing

<li>Merging Delivery day and Hours columns as Datetime to create date time object</li>

In [ ]:
timeMap = {
    f'H{i+1}': f'{i:02d}' for i in range(0, 24)
}

In [ ]:
df_price.loc[:, "Time"] = df_price["Hours"].apply(lambda x: timeMap[x] + ":00:00")

In [ ]:
df_price.loc[:, 'Datetime'] = df_price['Delivery Day'] + ' ' + df_price['Time']

<li>Changing data type of Datetime values to Timestamp</li>

In [ ]:
df_price.loc[:, 'Datetime'] = pd.to_datetime(df_price['Datetime'])

<li>Making Datetime Column as index</li>

In [ ]:
df_price.set_index('Datetime', inplace=True)

<li>Dropping Time Column</li>

In [ ]:
df_price.drop('Time', axis=1, inplace=True)

In [ ]:
df_price

#### 4.2 Visualizing Datas

<li>Time Series analysis of Daily Price</li>

In [ ]:
df_price['Prices\n(EUR/MWh)'].resample('D').mean().plot()
plt.xlabel('Date')
plt.ylabel('Average Price (EUR/MWh)')
plt.title('Daily Average Prices')
plt.show()

<li>Time Series Analysis of Weekly Price</li>

In [ ]:
df_price['Prices\n(EUR/MWh)'].resample('W').mean().plot()
plt.xlabel('Date')
plt.ylabel('Average Price (EUR/MWh)')
plt.title('Weekly Average Prices')
plt.show()

<li>Time Series Analysis of Monthly Price</li>

In [ ]:
df_price['Prices\n(EUR/MWh)'].resample('M').mean().plot()
plt.xlabel('Date')
plt.ylabel('Average Price (EUR/MWh)')
plt.title('Monthly Average Prices')
plt.show()


<li>Average Prices of electricity for each hour of day</li>

In [ ]:
hour_order = [f'H{i}' for i in range(1, 25)] 

# Group by the 'Hours' column and calculate the mean for each group
hourly_average = df_price.groupby('Hours')['Prices\n(EUR/MWh)'].mean()

plt.figure(figsize=(12, 6))

# Plot a bar graph
plt.bar(hour_order, hourly_average.reindex(hour_order))
plt.xlabel('Hour of the Day')
plt.ylabel('Average Price (EUR/MWh)')
plt.title('Average Hourly Prices (Bar Graph)')
plt.show()

### 5. Feature Engineering

#### [Zenml-MLfow pipeline for Feature Engineering](../steps/price_forecasting/feature_engineering.py)

#### 5.1 Normalization

- Normalize continuous values and avoid vanishing gradient problems to finalize our data before model training.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
df_price = scaler.fit_transform(df_price["Prices\n(EUR/MWh)"].values.reshape(-1,1))

In [ ]:
df_price

#### 5.2 Train Validation Test Split
- Splitting the data into different splits

In [ ]:
def train_test_split(df, train_size=0.8):
    df_train = df[:int(train_size * len(df))]
    df_test = df[int(train_size * len(df)):]
    return df_train, df_test

In [ ]:
df_train, df_test = train_test_split(df_price)

- Seperating dataset into train and test

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 15
X_train, y_train = create_dataset(dataset=df_train, time_step=time_step)
X_test, y_test = create_dataset(dataset=df_test, time_step=time_step)

In [ ]:
X_train.shape, y_train.shape

### 6. Model Evaluation and Training
#### [Zenml-MLfow pipeline for Model Evaluation](../steps/price_forecasting/model_evaluation.py)
#### [Zenml-MLfow pipeline for Model Training](../steps/price_forecasting/training.py)


#### 6.1 Making imports for Model Evaluation and Training

In [ ]:
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
from statsmodels.tsa.stattools import kpss
from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


#### 6.2 Model Training and Evaluation using XGBoost, LSTM, ARIMA and SARIMA

##### 6.2.1 XGBoost

- XGBoost model is configured with 1000 estimators and a learning rate of 0.05

In [ ]:
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
xgb_model.fit(X_train, y_train)

In [ ]:
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

- Evaluation of XGBoost model

In [ ]:
xgb_pred = xgb_model.predict(X_test)

In [ ]:
mean_squared_error(y_test, xgb_pred)

##### 6.2.2 LSTM

- The LSTM model is composed of an LSTM layer, followed by a dense layer with ReLU activation, a dropout layer, and a final dense layer.


In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(None, 1), return_sequences=True))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

- Training

In [ ]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

- Training and Validation Loss Visualization

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()


plt.show()

- Evaluation on Test Set

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

- Evaluation Metrics Calculation

In [ ]:
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

##### 6.2.3 ARIMA Model Order Selection
This script performs an exhaustive search for the best ARIMA model order (p, d, q) based on the Akaike Information Criterion (AIC). It evaluates multiple combinations of p, d, and q values to find the model order that minimizes the AIC.

In [ ]:
p_values = [0, 1, 2]
d_values = [0, 1]
q_values = [0, 1, 2]
pdq_combinations = [(p, d, q) for p in p_values for d in d_values for q in q_values]

def evaluate_arima_models(data, pdq_combinations):
    results = []
    for pdq in pdq_combinations:
        try:
            model = ARIMA(data, order=pdq)
            model_fit = model.fit()
            aic = model_fit.aic
            results.append((pdq, aic))
        except:
            continue
    return results

results = evaluate_arima_models(df_train, pdq_combinations)
best_aic = min(results, key=lambda x: x[1])[1]
best_pdq = [result[0] for result in results if result[1] == best_aic][0]


- ARIMA Model Fitting and Summary

In [ ]:
print("Best ARIMA parameters:", best_pdq)


In [ ]:
model = ARIMA(df_train, order=best_pdq)
model_fit = model.fit()

In [ ]:
model_fit.summary()

In [ ]:
df_predict = model_fit.predict(start=len(df_train), end=len(df_price)-1)

In [ ]:
mean_squared_error(df_test, df_predict)

#### 6.2.4 Time Series  Analysis and Seasonal Differencing 

In [ ]:

plot_acf(df_train, lags=40) 
plot_pacf(df_train, lags=40)

series_diff = pd.DataFrame(df_train).diff().dropna() 

plot_acf(series_diff, lags=40)
plot_pacf(series_diff, lags=40)

- KPSS Test for Seasonality:

In [ ]:

potential_m_values = [7, 12, 24]  
best_m = None
best_kpss_stat = float('inf')

for m in potential_m_values:
    series_diff = pd.DataFrame(df_train).diff(m).dropna()

    kpss_result = kpss(series_diff, regression='c')  
    kpss_stat = kpss_result[0]
    p_value = kpss_result[1]

    if kpss_stat < best_kpss_stat and p_value > 0.05:
        best_m = m
        best_kpss_stat = kpss_stat



In [ ]:
print("Best m value based on KPSS test:", best_m)

##### 6.2.5 SARIMA Model Order Selection

- The script iteratively fits SARIMA models with different seasonal order parameters and selects the configuration that minimizes the AIC.


In [ ]:


seasonal_order_values = range(3) 

best_aic = np.inf
best_seasonal_order = None

for seasonal_order_candidate in itertools.product(seasonal_order_values, repeat=3):
    p, d, q = (2, 1, 1)
    P, D, Q = seasonal_order_candidate
    m = 7 

    sarima_model = sm.tsa.SARIMAX(df_train, order=(p, d, q), seasonal_order=(P, D, Q, m))
    sarima_result = sarima_model.fit()

    current_aic = sarima_result.aic

    if current_aic < best_aic:
        best_aic = current_aic
        best_seasonal_order = (P, D, Q)

print("Best Seasonal Order:", best_seasonal_order)


In [ ]:
sarima_model = sm.tsa.SARIMAX(df_train, order=(2, 1, 1), seasonal_order=(1, 0, 2, 7))
sarima_result = sarima_model.fit()

In [ ]:
sarima_result.summary()

In [ ]:
df_predict = sarima_result.predict(start=len(df_train), end=len(df_price)-1)

#### 6.3 Evaluation
- After fitting the SARIMA model and obtaining predictions on the test set (`df_test`), the script calculates the Mean Squared Error (MSE) as an evaluation metric.


In [ ]:
mean_squared_error(df_test, df_predict)

- Insight : SARIMA model performed the best and was finallized for prediction

In [ ]:
joblib.dump(xgb_model, '../model/price_forecasting.pkl')